In [3]:
import pandas as pd
import numpy as np
import shap 
from sklearn.model_selection import train_test_split
import pickle 

In [4]:
# Reading in full data files
gene_expression = pd.read_csv(('/Users/christianlangridge/Desktop/Zhang-Lab/Zhang Lab Data/Full data files/Geneexpression (full).tsv'), sep='\t', header=0)
tf_expression = pd.read_csv(('/Users/christianlangridge/Desktop/Zhang-Lab/Zhang Lab Data/Full data files/TF(full).tsv'), sep='\t', header=0)

In [6]:
# Split into training, testing and validation sets and into numpy arrays + combining dataframes
x = tf_expression
y = gene_expression

combined_data = pd.concat([x, y], axis=1)

# First split: 70% train and 30% temp (test + val)
x_train, x_temp, y_train, y_temp = train_test_split(
    x, y, test_size=0.3, random_state=42)

# Second split: split the temp set into 20% test and 10% val (which is 2/3 and 1/3 of temp)
x_test, x_val, y_test, y_val = train_test_split(
    x_temp, y_temp, test_size=1/3, random_state=42)


# For training set
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()

# For validation set
x_val = x_val.to_numpy()
y_val = y_val.to_numpy()

# For testing set
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

In [7]:
# Load model in with pickle file 

with open('/Users/christianlangridge/Desktop/Zhang-Lab/Zhang Lab Data/Saved models/Random Forest/RF_model.pkl', 'rb') as file:
    loaded = pickle.load(file)
    
# Evaluate model 

# If you saved a list of estimators (one per gene) build predictions matrix

if isinstance(loaded, list):
    models = loaded
    print("Loaded object is a list of models. n_models:", len(models))
    y_pred = np.column_stack([m.predict(x_test) for m in models])  # (n_samples, n_genes)
else:
    model = loaded
    print("Loaded object type:", type(model))
    # If single-output model but y_test is multi-column, this will raise -- handled later
    y_pred = model.predict(x_test)

print("y_pred.shape:", y_pred.shape)

# diagnostics (safe access)
def safe_attr(obj, name):
    return getattr(obj, name, None) if not isinstance(obj, list) else None

print("model.n_features_in_:", safe_attr(loaded, "n_features_in_"))
print("model.n_outputs_:", safe_attr(loaded, "n_outputs_"))
print("x_test.shape:", x_test.shape)
print("y_test.shape:", y_test.shape)

# Continue with your R^2 / MSE logic expecting y_pred shape (n_samples, n_targets)
from sklearn.metrics import r2_score, mean_squared_error

# If y_pred is 1D, treat as single-output
if y_pred.ndim == 1 or (y_pred.ndim == 2 and y_pred.shape[1] == 1):
    if y_test.ndim == 2 and y_test.shape[1] > 1:
        raise ValueError(
            "Model predicts a single target but y_test contains multiple targets (genes).\n"
            "Select the trained target column before splitting, e.g.:\n"
            "  target = 'GENE_NAME'\n"
            "  y = gene_expression[target]\n"
            "  then redo train_test_split and evaluation."
        )
    y_true = y_test.ravel()
    print("R^2:", r2_score(y_true, y_pred))
    print("MSE:", mean_squared_error(y_true, y_pred))
else:
    print("Multi-output R^2 (uniform_average):", r2_score(y_test, y_pred, multioutput='uniform_average'))


Loaded object is a list of models. n_models: 16101
y_pred.shape: (3187, 16101)
model.n_features_in_: None
model.n_outputs_: None
x_test.shape: (3187, 1198)
y_test.shape: (3187, 16101)
Multi-output R^2 (uniform_average): 0.7601639389405871


In [8]:
first = models[0]
print("first estimator type:", type(first))
print("n_features_in_:", getattr(first, "n_features_in_", None))
print("example feature_importances_ (first 10):", getattr(first, "feature_importances_", None)[:10])

first estimator type: <class 'xgboost.sklearn.XGBRFRegressor'>
n_features_in_: 1198
example feature_importances_ (first 10): [0.02488494 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
